In [1]:
from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings('ignore')

In [2]:
spark = SparkSession.builder.master('local').appName('duplicate').getOrCreate()

21/07/14 02:16:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/07/14 02:16:27 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
dataF = spark.createDataFrame([('1','2400','32','9','3','PHILIPS','1XZFG')
            , ('2','2400','23','13','3','Apple','2XZFG')
            , ('3','2400','22','2.5','0','LG','3XZFG')
            , ('4','2400','33','16.5','3','None','4XZFG')
            , ('8','3500','33','23','5.5','LG','8XZFG')
            , ('6','2400','23','9','0','SONY','6XZFG')
            , ('7','3500','33','23','5.5','PHILIPS','7XZFG')
            , ('8','3500','33','23','5.5','LG','8XZFG')
            , ('9','2400','22','2.5','0','Apple','9XZFG')
            , ('18','2400','22','2.5','0','Apple','9XZFG')
            , ('10','2400','23','6','0','None','10XZFG')
            , ('15','2450','23','8','1','SONY','DCEZX15')
            , ('11','1200','23','6','2','SONY','11XZFG')
            , ('12','3500','23','6','2','LG','12XZFG')
            , ('17','3500','23','6','2','LG','12XZFG')
            , ('13','2400','33','16.5','5.5','Apple','13XZFG')
            , ('14','3500','23','6','0','DELL','14XZFG')
            , ('15','2400','22','6','1','SONY','15XZFG')],
['Id','Salary','age','experience','Coef','TV ','CODE_ID'])

In [4]:
dataF.show()

+---+------+---+----------+----+-------+-------+
| Id|Salary|age|experience|Coef|    TV |CODE_ID|
+---+------+---+----------+----+-------+-------+
|  1|  2400| 32|         9|   3|PHILIPS|  1XZFG|
|  2|  2400| 23|        13|   3|  Apple|  2XZFG|
|  3|  2400| 22|       2.5|   0|     LG|  3XZFG|
|  4|  2400| 33|      16.5|   3|   None|  4XZFG|
|  8|  3500| 33|        23| 5.5|     LG|  8XZFG|
|  6|  2400| 23|         9|   0|   SONY|  6XZFG|
|  7|  3500| 33|        23| 5.5|PHILIPS|  7XZFG|
|  8|  3500| 33|        23| 5.5|     LG|  8XZFG|
|  9|  2400| 22|       2.5|   0|  Apple|  9XZFG|
| 18|  2400| 22|       2.5|   0|  Apple|  9XZFG|
| 10|  2400| 23|         6|   0|   None| 10XZFG|
| 15|  2450| 23|         8|   1|   SONY|DCEZX15|
| 11|  1200| 23|         6|   2|   SONY| 11XZFG|
| 12|  3500| 23|         6|   2|     LG| 12XZFG|
| 17|  3500| 23|         6|   2|     LG| 12XZFG|
| 13|  2400| 33|      16.5| 5.5|  Apple| 13XZFG|
| 14|  3500| 23|         6|   0|   DELL| 14XZFG|
| 15|  2400| 22|    

In [5]:
dataF.count(), dataF.distinct().count()

(18, 17)

In [6]:
all_deleted = dataF.dropDuplicates()

In [7]:
(dataF.groupBy(dataF.columns).count().filter('count > 1').show())

+---+------+---+----------+----+---+-------+-----+
| Id|Salary|age|experience|Coef|TV |CODE_ID|count|
+---+------+---+----------+----+---+-------+-----+
|  8|  3500| 33|        23| 5.5| LG|  8XZFG|    2|
+---+------+---+----------+----+---+-------+-----+



In [9]:
(all_deleted.groupBy([col for col in all_deleted.columns if col != 'Id']).count().filter('count > 1').show())

+------+---+----------+----+-----+-------+-----+
|Salary|age|experience|Coef|  TV |CODE_ID|count|
+------+---+----------+----+-----+-------+-----+
|  2400| 22|       2.5|   0|Apple|  9XZFG|    2|
|  3500| 23|         6|   2|   LG| 12XZFG|    2|
+------+---+----------+----+-----+-------+-----+



In [10]:
id_deleted = all_deleted.dropDuplicates(subset = [col for col in all_deleted.columns if col != 'Id'])

In [12]:
id_deleted.count(), id_deleted.distinct().count()

(15, 15)

In [14]:
import pyspark.sql.functions as sqlfunct

In [15]:
id_deleted.agg(
    sqlfunct.count('Id').alias('Count_Ids'),
    sqlfunct.countDistinct('Id').alias('Count_Distinct_Ids')
).show()

+---------+------------------+
|Count_Ids|Count_Distinct_Ids|
+---------+------------------+
|       15|                14|
+---------+------------------+



In [16]:
(id_deleted.groupBy('Id').count().filter('count > 1').show())

+---+-----+
| Id|count|
+---+-----+
| 15|    2|
+---+-----+



In [17]:
new_id = (
    id_deleted.select(
        [sqlfunct.monotonically_increasing_id().alias('Id')] + 
        [col for col in id_deleted.columns if col != 'Id']
    )
)

In [18]:
new_id.show()

+-------------+------+---+----------+----+-------+-------+
|           Id|Salary|age|experience|Coef|    TV |CODE_ID|
+-------------+------+---+----------+----+-------+-------+
|  25769803776|  3500| 23|         6|   2|     LG| 12XZFG|
|  51539607552|  3500| 33|        23| 5.5|PHILIPS|  7XZFG|
| 249108103168|  2400| 32|         9|   3|PHILIPS|  1XZFG|
| 412316860416|  2400| 22|       2.5|   0|  Apple|  9XZFG|
| 498216206336|  2400| 23|         6|   0|   None| 10XZFG|
| 566935683072|  2400| 23|        13|   3|  Apple|  2XZFG|
| 704374636544|  1200| 23|         6|   2|   SONY| 11XZFG|
| 721554505728|  2400| 33|      16.5|   3|   None|  4XZFG|
| 893353197568|  3500| 23|         6|   0|   DELL| 14XZFG|
| 962072674304|  2400| 33|      16.5| 5.5|  Apple| 13XZFG|
| 970662608896|  2400| 22|         6|   1|   SONY| 15XZFG|
|1133871366144|  3500| 33|        23| 5.5|     LG|  8XZFG|
|1142461300736|  2400| 23|         9|   0|   SONY|  6XZFG|
|1503238553600|  2400| 22|       2.5|   0|     LG|  3XZF